# Analiza mreže javnog gradskog prevoza u Beogradu

### Opis grafa
TODO

Podešavanje okruženja:
+ pip install ipykernel
+ pip install numpy scipy networkx pandas matplotlib
+ pip install scikit-learn python-docx

Linkovi:
+ [Portal otvorenih podataka](https://data.gov.rs/sr/datasets/gtfs/)
+ [General Transit Feed Specification](https://en.wikipedia.org/wiki/GTFS)
+ [Linije javnog prevoza u Beogradu](https://sr.wikipedia.org/sr-el/%D0%9B%D0%B8%D0%BD%D0%B8%D1%98%D0%B5_%D1%98%D0%B0%D0%B2%D0%BD%D0%BE%D0%B3_%D0%BF%D1%80%D0%B5%D0%B2%D0%BE%D0%B7%D0%B0_%D1%83_%D0%91%D0%B5%D0%BE%D0%B3%D1%80%D0%B0%D0%B4%D1%83)
+ [Sekretarijat za javni prevoz](https://www.bgprevoz.rs/)

### Globalne stvari

In [ ]:
# importi
from datetime import datetime, date, timedelta
import numpy as np
import scipy.stats as stats
import networkx as nx
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from itertools import chain, combinations
from sklearn.cluster import SpectralClustering
from scipy.cluster.hierarchy import dendrogram


# putanje
original_path = "../data/!original/"
clean_path = "../data/clean/"

# globalne funkcije

def series_to_single_value(series):
    return series.to_list()[0]

def cleaned_null_columns(df:pd.DataFrame):
    def column_has_values(column):
        return True
    non_null_columns = [column for column in df.columns if not df[column].isna().all()]
    return df[non_null_columns]

def cleaned_duplicates(df:pd.DataFrame):
    no_duplicates = df.drop_duplicates()
    number_of_duplicates = df.shape[0] - no_duplicates.shape[0]
    return no_duplicates

def cleaned_data_frame(df:pd.DataFrame):
    return cleaned_duplicates(cleaned_null_columns(df))


def is_valid_datetime_str(datetime_str):
    format_str = "%H:%M:%S"
    try:
        datetime.strptime(datetime_str, format_str)
        return True
    except ValueError:
        return False
    except TypeError:
        return False

def midnight_converter(datetime_str):
    if (not str.startswith(datetime_str, "24")):
        return datetime_str
    else:
        return "00" + datetime_str[2:]
# TODO

### Čišćenje podataka

In [ ]:
# # # Originalni podaci se čuvaju u data/!original/ folderu.
# Prečišćeni podaci se čuvaju u data/clean/ folderu.
# BgNetLWalk podaci se čuvaju u data/consolidate/ folderu.    

# TODO: izabrati koji se podaci koriste za izvršavanje
# ! Komentare koji pocinju sa uzvicnikom ili upitnikom izbaciti iz finalne verzije
df_agency = pd.read_csv(f"{original_path}/agency.csv")
df_calendar_dates = pd.read_csv(f"{original_path}/calendar_dates.csv")
df_calendar = pd.read_csv(f"{original_path}/calendar.csv")
df_fare_attributes = pd.read_csv(f"{original_path}/fare_attributes.csv")
df_fare_rules = pd.read_csv(f"{original_path}/fare_rules.csv")
df_feed_info = pd.read_csv(f"{original_path}feed_info.csv")
df_frequencies = pd.read_csv(f"{original_path}/frequencies.csv")#? Treba li da radimo u opstem slucaju i pravimo da ova tabela postoji, ili je bas poenta da je ignorisemo
df_routes = pd.read_csv(f"{original_path}/routes.csv")
df_shapes = pd.read_csv(f"{original_path}/shapes.csv")
df_stops = pd.read_csv(f"{original_path}/stops.csv")
df_stop_times = pd.read_csv(f"{original_path}/stop_times.csv", dtype={"stop_headsign":np.string_})
df_transfers = pd.read_csv(f"{original_path}/transfers.csv")
df_trips = pd.read_csv(f"{original_path}/trips.csv")




all_frames = [df_agency, df_calendar_dates, df_calendar, df_fare_attributes, df_fare_rules, df_feed_info, df_frequencies, df_routes, df_shapes, df_stops, df_stop_times, df_transfers, df_trips]
all_frames_cleaned = [cleaned_data_frame(df) for df in all_frames]




In [ ]:
[df_agency, df_calendar_dates, df_calendar, df_fare_attributes, df_fare_rules, df_feed_info, df_frequencies, df_routes, df_shapes, df_stops, df_stop_times, df_transfers, df_trips] = all_frames_cleaned
df_agency.to_csv(f"{clean_path}/agency.csv", index=False)
df_calendar_dates.to_csv(f"{clean_path}/calendar_dates.csv", index=False)
df_calendar.to_csv(f"{clean_path}/calendar.csv", index=False)
df_fare_attributes.to_csv(f"{clean_path}/fare_attributes.csv", index=False)
df_fare_rules.to_csv(f"{clean_path}/fare_rules.csv", index=False)
df_feed_info.to_csv(f"{clean_path}/feed_info.csv", index=False)
df_frequencies.to_csv(f"{clean_path}/frequencies.csv", index=False)
df_routes.to_csv(f"{clean_path}/routes.csv", index=False)
df_shapes.to_csv(f"{clean_path}/shapes.csv", index=False)
df_stops.to_csv(f"{clean_path}/stops.csv", index=False)
df_stop_times.to_csv(f"{clean_path}/stop_times.csv", index=False)
df_transfers.to_csv(f"{clean_path}/transfers.csv", index=False)
df_trips.to_csv(f"{clean_path}/trips.csv", index=False)

In [ ]:
df_agency = pd.read_csv(f"{clean_path}/agency.csv")
df_calendar_dates = pd.read_csv(f"{clean_path}/calendar_dates.csv")
df_calendar = pd.read_csv(f"{clean_path}/calendar.csv")
df_fare_attributes = pd.read_csv(f"{clean_path}/fare_attributes.csv")
df_fare_rules = pd.read_csv(f"{clean_path}/fare_rules.csv")
df_feed_info = pd.read_csv(f"{clean_path}/feed_info.csv")
df_routes = pd.read_csv(f"{clean_path}/routes.csv")
df_shapes = pd.read_csv(f"{clean_path}/shapes.csv")
df_stops = pd.read_csv(f"{clean_path}/stops.csv")
df_stop_times = pd.read_csv(f"{clean_path}/stop_times.csv", dtype={"stop_headsign":np.string_})
df_trips = pd.read_csv(f"{clean_path}/trips.csv")


#print(df_routes["route_type"].unique()) #rezultati (0,3, 702, 800) 0 - tramvaj, 3 - autobus, 702 - ekspres autobus, 800 - trolejbus
route_type_names = {0:"Tramvaj", 3:"Autobus", 702:"Autobus", 800:"Trolejbus"}
df_routes["route_type_name"] = df_routes["route_type"].map(route_type_names)

In [ ]:
df_stop_times = df_stop_times.loc[df_stop_times["arrival_time"].notna() & df_stop_times["departure_time"].notna()]
df_stop_times["arrival_time"] = df_stop_times["arrival_time"].map(lambda x: midnight_converter(x))
df_stop_times["departure_time"] = df_stop_times["departure_time"].map(lambda x: midnight_converter(x))
#for dep_time in df_stop_times["departure_time"]:
    #if (not is_valid_datetime_str(dep_time)):
        #print(f"Non valid {dep_time}")
is_valid_datetime_str(123)

### Statistička obrada podataka [5 poena]
1) Odrediti prosečno vreme između dva polaska svake od linija. Koje su linije sa najčešćim, a koje sa najređim polascima?
2) Koje su linije sa najvećim brojem stanica, a koje stanice sa najvećim brojem linija?
3) Preračunati očekivani dnevni promet stanice kao ukupan očekivani broj stajanja linije na toj stanici, određen na osnovu planiranog rasporeda vožnje. Koje su stanice sa najvećim očekivanim dnevnim prometom?
4) Svakoj stanici dodeliti tip na osnovu vrsta prevoza koje prolaze njom (stanica autobusa, tramvaja, trolejbusa ili meštovita stanica). Koliko ima stanica svakog tipa?

In [71]:
# 1) Odrediti prosečno vreme između dva polaska svake od linija.
# Koje su linije sa najčešćim, a koje sa najređim polascima?

# TODO
def find_average_departure_times_for_stops():
    def get_average_wait(departure_times_str):
        def convert_to_time(time_str):
            time_object = datetime.strptime(time_str, '%H:%M:%S').time()
            return time_object
        departure_times = [convert_to_time(time_str) for time_str in departure_times_str if is_valid_datetime_str(time_str)]
        departure_times = sorted(departure_times)
        average_timedelta = timedelta(0)
        num_elements = len(departure_times)
        for index in range(1, len(departure_times)):
            prev = index - 1
            duration = datetime.combine(date.min, departure_times[index]) - datetime.combine(date.min, departure_times[prev])
            average_timedelta = average_timedelta + duration  
        if num_elements == 0:
            inf = 25*60
            return pd.NA
        average_timedelta = average_timedelta / num_elements
        return average_timedelta 
    
    route_names_dict = df_routes[["route_id", "route_short_name"]].to_dict()

    stop_times_with_route_id = pd.merge(df_stop_times[["stop_id", "trip_id", "departure_time"]], df_trips[["trip_id", "route_id", "service_id"]], how="inner", on="trip_id")[["stop_id", "departure_time", "route_id", "service_id"]]
    stop_times_with_route_id_grouped = stop_times_with_route_id.groupby(["stop_id", "route_id", "service_id"])["departure_time"].apply(lambda x: get_average_wait(list(x)))
    stop_times_with_route_id_waits = pd.DataFrame(stop_times_with_route_id_grouped)
    stop_times_with_route_id_waits_sorted = stop_times_with_route_id_waits.sort_values("departure_time")
    stop_times_with_route_id_waits_sorted = stop_times_with_route_id_waits_sorted.rename(columns={"departure_time":"average_wait"})
    stop_times_with_route_id_waits_sorted = stop_times_with_route_id_waits_sorted.dropna(subset=["average_wait"])
    stop_times_with_route_id_waits_sorted = stop_times_with_route_id_waits_sorted[stop_times_with_route_id_waits_sorted["average_wait"] != timedelta(0)]#radi za sad
    
    stop_times_with_route_names_waits_sorted = stop_times_with_route_id_waits_sorted 
    #TODO Zasto ovo ne radi 
    #stop_times_with_route_names_waits_sorted["route_id"] = stop_times_with_route_names_waits_sorted["route_id"].map(lambda x: route_names_dict[x])
    
    return stop_times_with_route_id_waits_sorted

find_average_departure_times_for_stops()

average_wait
stop_id route_id service_id                      
2124    50037    Zimski-Nedelja           0:01:00
                 Zimski-Radni Dan         0:01:00
                 Zimski-Subota            0:01:00
333     50015    Zimski-Nedelja           0:01:30
1576    50308    Zimski-Radni Dan         0:01:30
...                                           ...
404     612      Zimski-Nedelja           1:32:00
871     54       Zimski-Nedelja    1:43:46.153846
                 Zimski-Subota     1:43:46.153846
873     54       Zimski-Subota     1:43:50.769231
                 Zimski-Nedelja    1:43:50.769231

[24147 rows x 1 columns]

In [ ]:
# 2) Koje su linije sa najvećim brojem stanica, a koje stanice sa najvećim brojem linija?

# TODO
#stops (identified by the name) with routes

# TODO Treba li izvrsiti spajanje stanica pa iskombinovati sa ovim?
# TODO: Malko su sumnjivi rezultati
def find_routes_with_most_stops():
    stop_times_with_route_id = pd.merge(df_stop_times[["stop_id", "trip_id"]], df_trips[["trip_id", "route_id"]], how="inner", on="trip_id")
    route_ids_with_num_stops = stop_times_with_route_id.groupby("route_id")["stop_id"].nunique().reset_index(name='number_of_distinct_stop_ids')
    route_ids_with_max_stops = route_ids_with_num_stops.loc[route_ids_with_num_stops["number_of_distinct_stop_ids"] == route_ids_with_num_stops["number_of_distinct_stop_ids"].max()]
    route_with_most_stops_with_names = pd.merge(df_routes[["route_id", "route_short_name"]], route_ids_with_max_stops, how="inner", on="route_id")
    return route_with_most_stops_with_names


def find_stops_with_most_routes():
    stop_times_with_route_id = pd.merge(df_stop_times[["stop_id", "trip_id"]], df_trips[["trip_id", "route_id"]], how="inner", on="trip_id")
    stop_ids_with_num_routes = stop_times_with_route_id.groupby("stop_id")["route_id"].nunique().reset_index(name='number_of_distinct_route_ids')
    stops_with_most_routes = stop_ids_with_num_routes.loc[stop_ids_with_num_routes["number_of_distinct_route_ids"] == stop_ids_with_num_routes["number_of_distinct_route_ids"].max()]
    stops_with_most_routes_with_names = pd.merge(df_stops[["stop_id", "stop_name"]], stops_with_most_routes, how="inner", on="stop_id")
    return stops_with_most_routes_with_names

#find_stops_with_most_routes()
find_routes_with_most_stops()




In [ ]:
# 3) Preračunati očekivani dnevni promet stanice kao ukupan očekivani broj stajanja linije na toj stanici,
# određen na osnovu planiranog rasporeda vožnje. Koje su stanice sa najvećim očekivanim dnevnim prometom?

# TODO: Veliko je pitanje da li se ovi tripovi upisuju svakoga dana ili, kao sto cu za sad pretpostaviti, 
# planirani generalno za dan, tj. da ne moram da delim sa brojem dana.  



def get_stops_with_most_traffic():
    stops_with_trips = pd.merge(df_stop_times[["stop_id", "trip_id"]], df_trips[["trip_id", "service_id"]], how="inner", on="trip_id")
    #stops_with_trips = pd.merge(stop_ids_with_trips, df_stops[["stop_name", "stop_id"]], how="inner", on="stop_id")[["stop_name", "service_id", "trip_id"]]
    stops_with_trips_grouped = stops_with_trips.groupby(["stop_id", "service_id"]).nunique()
    stops_with_trips_sorted =  stops_with_trips_grouped.sort_values("trip_id", ascending=False)
    stops_with_trips_sorted_with_names = pd.merge(df_stops[["stop_name", "stop_id"]], stops_with_trips_sorted, on="stop_id", how= "inner").sort_values("trip_id", ascending=False)
    return stops_with_trips_sorted_with_names[["stop_name", "trip_id"]].rename(columns={"trip_id":"Number of trips"})

get_stops_with_most_traffic()

In [ ]:
# 4) Svakoj stanici dodeliti tip na osnovu vrsta prevoza koje prolaze njom
# (stanica autobusa, tramvaja, trolejbusa ili meštovita stanica). Koliko ima stanica svakog tipa?

# TODO Da li je ovde bitan smer?
def get_stops_with_route_type():
    stops_with_route_id = pd.merge(df_stop_times[["stop_id", "trip_id"]], df_trips[["trip_id", "route_id"]], how="inner", on="trip_id")
    stops_with_route_type = pd.merge(stops_with_route_id[["stop_id", "route_id"]], df_routes[["route_id", "route_type_name"]], how="inner", on="route_id")[["stop_id", "route_type_name"]]
    stops_with_route_type = pd.DataFrame(stops_with_route_type.groupby('stop_id')['route_type_name'].apply(lambda x: ', '.join(x.unique())))
    stops_with_route_type.loc[stops_with_route_type['route_type_name'].str.contains(","), 'route_type_name'] = "Mesano"
    return stops_with_route_type

def count_stops_per_route_type(stops_with_route_type:pd.DataFrame):
    route_types_with_num_stops = stops_with_route_type.groupby("route_type_name")["stop_id"].nunique().reset_index(name="Number of stops")
    return route_types_with_num_stops


stops_with_route_type = get_stops_with_route_type()
stops_with_route_type_full = pd.merge(df_stops, stops_with_route_type, how="inner", on="stop_id")
stops_with_route_type_full.to_csv(f"{clean_path}/stops_with_route_type.csv", index=False)
count_stops_per_route_type(stops_with_route_type_full)



### Osnovna karakterizacija modelovanih mreža [10 poena]
5) Kolika je gustina mreže?
6) Kolike su prosečne distance u okviru mreže i dijametar mreže?
7) U kojoj meri je mreža povezana i centralizovana? Navesti broj i veličine povezanih komponenata i proceniti da li postoji gigantska komponenta.
8) Koliki je prosečni, a koliki globalni koeficijent klasterizacije mreže? Kakva je raspodela lokalnog koeficijenta klasterizacije njenih čvorova? Da li je klasterisanje izraženo ili ne? Odgovor dati upoređivanjem sa slučajno generisanim Erdos-Renyi i scale free mrežama istih dimenzija.
9) Na osnovu odgovora na pitanja 6 i 8, proceniti da li mreža iskazuje osobine malog sveta.
10) Izvršiti asortativnu analizu po stepenu čvora i dati odgovor da li je izraženo asortativno mešanje. Priložiti i vizuelizaciju.
11) Da li mreža ispoljava fenomen kluba bogatih (eng. rich club phenomenon)?
12) Analizirati stajališta na osnovu tarifne zone kojoj pripadaju. Ispitati da li BGNetL i BGNetP mreže odgovaraju modelu jezgra i periferije (core-periphery model). Smatrati da prva zona pripada jezgru, a ostale periferiji.
13) Kakva je distribucija čvorova po stepenu i da li prati power law raspodelu?

In [ ]:
# 5) Kolika je gustina mreže?

#TODO

In [ ]:
# 6) Kolike su prosečne distance u okviru mreže i dijametar mreže?

#TODO

In [ ]:
# 7) U kojoj meri je mreža povezana i centralizovana?
# Navesti broj i veličine povezanih komponenata i proceniti da li postoji gigantska komponenta.

#TODO

In [ ]:
# 8) Koliki je prosečni, a koliki globalni koeficijent klasterizacije mreže?
# Kakva je raspodela lokalnog koeficijenta klasterizacije njenih čvorova?
# Da li je klasterisanje izraženo ili ne?
# Odgovor dati upoređivanjem sa slučajno generisanim Erdos-Renyi i scale free mrežama istih dimenzija.

#TODO

In [ ]:
# 9) Na osnovu odgovora na pitanja 6 i 8, proceniti da li mreža iskazuje osobine malog sveta.

#TODO

In [ ]:
# 10) Izvršiti asortativnu analizu po stepenu čvora i dati odgovor
# da li je izraženo asortativno mešanje. Priložiti i vizuelizaciju.

#TODO

In [ ]:
# 11) Da li mreža ispoljava fenomen kluba bogatih (eng. rich club phenomenon)?

#TODO

In [ ]:
# 12) Analizirati stajališta na osnovu tarifne zone kojoj pripadaju.
# Ispitati da li BGNetL i BGNetP mreže odgovaraju modelu jezgra i periferije (core-periphery model).
# Smatrati da prva zona pripada jezgru, a ostale periferiji.

#TODO

In [ ]:
# 13) Kakva je distribucija čvorova po stepenu i da li prati power law raspodelu?

#TODO

### Analiza mera centralnosti [5 poena]
14) Sprovesti analize centralnosti po stepenu, bliskosti i relacionoj centralnosti. Dati pregled najvažnijih aktera po svakoj od njih.
15) Ko su najvažniji akteri po centralnosti po sopstvenom vektoru? Šta nam to govori o njima?
16) Na osnovu prethodna dva pitanja predložiti i konstruisati heuristiku (kompozitnu meru centralnosti) za pronalaženje najvažnijih aktera i pronaći ih. Obratiti pažnju na tip mreže koji se analizira (usmerena ili neusmerena) i, shodno tome, prilagoditi koliko različite mrežne metrike utiču na heuristiku.

In [ ]:
# 14) Sprovesti analize centralnosti po stepenu, bliskosti i relacionoj centralnosti.
# Dati pregled najvažnijih aktera po svakoj od njih.

# TODO

In [ ]:
# 15) Ko su najvažniji akteri po centralnosti po sopstvenom vektoru? Šta nam to govori o njima?

# TODO

In [ ]:
# 16) Na osnovu prethodna dva pitanja predložiti i konstruisati heuristiku (kompozitnu meru centralnosti)
# za pronalaženje najvažnijih aktera i pronaći ih. Obratiti pažnju na tip mreže koji se analizira (usmerena ili neusmerena) i,
# shodno tome, prilagoditi koliko različite mrežne metrike utiču na heuristiku.

# TODO

### Detekcija komuna Luvenskom metodom [5 poena]
17) Sprovesti klasterisanje Luvenskom metodom (maksimizacijom modularnosti) u alatu Gephi za tri različite vrednosti parametra rezolucije. Konstruisati vizuelizacije i diskutovati izbor parametra rezolucije na dobijeno klasterisanje (broj i veličina klastera).
18) Koje zajednice (komune) se mogu uočiti prilikom analize mreže, a koji akteri su ključni brokeri? Da li postoji neko objašnjenje za detektovane komune? Odgovor dati posmatrajući ne samo strukturu mreže, već i atribute preračunate u pitanjima 3 i 4.

In [ ]:
# 17) Sprovesti klasterisanje Luvenskom metodom (maksimizacijom modularnosti) u alatu Gephi za tri različite vrednosti parametra rezolucije.
# Konstruisati vizuelizacije i diskutovati izbor parametra rezolucije na dobijeno klasterisanje (broj i veličina klastera).

# TODO

In [ ]:
# 18) Koje zajednice (komune) se mogu uočiti prilikom analize mreže, a koji akteri su ključni brokeri?
# Da li postoji neko objašnjenje za detektovane komune?
# Odgovor dati posmatrajući ne samo strukturu mreže, već i atribute preračunate u pitanjima 3 i 4.

# TODO

### Detekcija komuna spektralnim klasterisanjem [10 poena]
19) Predložiti funkcije sličnosti za L i P modele koje, osim strukturnih informacija iz odgovarajućih mreža, uključuju i geografske podatke o čvorovima i konstruisati matrice sličnosti i graf Laplasijane na osnovu predloženih funkcija sličnosti.
20) Sprovesti sprektralnu analizu i proceniti potencijalne kandidate za broj komuna u mreži. Uporediti rezultat sa dendogramom konstruisanim Girvan-Newman metodom.
21) Izvršiti spektralno klasterisanje na osnovu konstruisanih funkcija sličnosti i procenjenog broja klastera. Vizuelizovati klasterisanje na mapi gradskog saobraćaja, tako što će se stanice koje su svrstane u isti klaster crtati istom bojom. Izabrati boje koje daju dobar kontrast u odnosu na mapu i boje drugih klastera.
22) Ko su akteri koji se mogu okarakterisati kao ključni brokeri (mostovi) u mreži? Šta ih čini brokerima? Porediti odgovor sa brokerima dobijenim u pitanju 17.

In [ ]:
# 19) Predložiti funkcije sličnosti za L i P modele koje, osim strukturnih informacija iz odgovarajućih mreža,
# uključuju i geografske podatke o čvorovima i konstruisati matrice sličnosti i graf Laplasijane na osnovu predloženih funkcija sličnosti.

# TODO

In [ ]:
# 20) Sprovesti sprektralnu analizu i proceniti potencijalne kandidate za broj komuna u mreži.
# Uporediti rezultat sa dendogramom konstruisanim Girvan-Newman metodom.

# TODO

In [ ]:
# 21) Izvršiti spektralno klasterisanje na osnovu konstruisanih funkcija sličnosti i procenjenog broja klastera.
# Vizuelizovati klasterisanje na mapi gradskog saobraćaja, tako što će se stanice koje su svrstane u isti klaster crtati istom bojom.
# Izabrati boje koje daju dobar kontrast u odnosu na mapu i boje drugih klastera.

# TODO

In [ ]:
# 22) Ko su akteri koji se mogu okarakterisati kao ključni brokeri (mostovi) u mreži?
# Šta ih čini brokerima?
# Porediti odgovor sa brokerima dobijenim u pitanju 17.

# TODO

### Analiza mreže BGNetLWalk [5 poena]
23) Ponoviti sve analize koje imaju smisla za BGNetLWalk mrežu. Uporediti i komentarisati dobijene rezultate sa BGNetL mrežom. Da li se uočavaju značajnije promene u mreži?

In [ ]:
# 23) Ponoviti sve analize koje imaju smisla za BGNetLWalk mrežu.
# Uporediti i komentarisati dobijene rezultate sa BGNetL mrežom.
# Da li se uočavaju značajnije promene u mreži?

# TODO: pokrenuti notebook sa drugačijim parametrima